## Multi-Label Classifier with Feed-Forward Nural Netwark

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
import pandas as pd
import numpy as np

In [3]:
CICIDS_data = pd.read_csv('../Wednesday-workingHours.pcap_ISCX.csv')

In [4]:
# dropping duplications
def find_duplicated_columns(df):
    grouped_columns = []
    seen_columns = set()
    for col in df.columns:
        if col not in seen_columns:
            duplicates = [col] + [c for c in df.columns if c != col and df[col].equals(df[c])]
            if len(duplicates)>1:
                grouped_columns.extend(duplicates[1:])
            seen_columns.update(duplicates)
    return grouped_columns

In [5]:
# data preproccessing 
CICIDS_data.columns = [col.strip() for col in CICIDS_data.columns]
CICIDS_data.drop(CICIDS_data.columns[CICIDS_data.eq(0).all()], axis=1, inplace=True)
CICIDS_data.drop(find_duplicated_columns(CICIDS_data), axis=1, inplace=True)

In [6]:
# splitting my data into features (X) and target (y) ignoring the columns with inf values
X = CICIDS_data.drop(['Label','Flow Bytes/s', 'Flow Packets/s'],axis=1).copy()
y = CICIDS_data['Label'].copy()

In [7]:
# filling nan values with -1
X.fillna(-1, inplace=True)

In [8]:
# scaling my features
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# encoding my categorical labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [10]:
# splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [11]:
# building the model with 3 hidden layers
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

2024-07-18 13:24:21.882638: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
# compailing the model with hyper parameters
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
# training the model with the training set
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

2024-07-18 13:24:22.031883: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
13855/13855 [==============================] - 12s 806us/step - loss: 363.9926 - accuracy: 0.9158 - val_loss: 0.1837 - val_accuracy: 0.9601
Epoch 2/20
13855/13855 [==============================] - 11s 805us/step - loss: 0.3126 - accuracy: 0.9613 - val_loss: 0.1592 - val_accuracy: 0.9621
Epoch 3/20
13855/13855 [==============================] - 11s 761us/step - loss: 0.4988 - accuracy: 0.9619 - val_loss: 0.1989 - val_accuracy: 0.9624
Epoch 4/20
13855/13855 [==============================] - 12s 860us/step - loss: 0.2791 - accuracy: 0.9210 - val_loss: 0.2771 - val_accuracy: 0.8993
Epoch 5/20
13855/13855 [==============================] - 11s 825us/step - loss: 0.3165 - accuracy: 0.8992 - val_loss: 0.2704 - val_accuracy: 0.9018
Epoch 6/20
13855/13855 [==============================] - 12s 843us/step - loss: 0.3855 - accuracy: 0.8994 - val_loss: 0.2733 - val_accuracy: 0.8997
Epoch 7/20
13855/13855 [==============================] - 11s 798us/step - loss: 0.7674 - accuracy: 0.89

In [14]:
# predicting the classes in the test set
y_pred = np.argmax(model.predict(X_test), axis=1)

In [17]:
# evaluating the model performance 
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print(confusion_matrix(y_test, y_pred))

                  precision    recall  f1-score   support

          BENIGN       0.87      1.00      0.93     88170
   DoS GoldenEye       0.99      0.58      0.74      2017
        DoS Hulk       1.00      0.77      0.87     46147
DoS Slowhttptest       0.88      0.12      0.21      1090
   DoS slowloris       0.96      0.05      0.09      1114
      Heartbleed       0.00      0.00      0.00         3

        accuracy                           0.90    138541
       macro avg       0.78      0.42      0.47    138541
    weighted avg       0.91      0.90      0.89    138541



/Users/traviv/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/traviv/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/traviv/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NameError: name 'confusion_matrix' is not defined